In [1]:
# Installation des librairies nécessaires
!pip install tensorflow pillow scipy

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import resample
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Input

# --- CONFIGURATION DES CHEMINS ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
PATH_IMG_TRAIN = os.path.join(PROJECT_ROOT, "images", "images", "image_train")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")

# Création du dossier de sortie si ça bug
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📂 Racine projet : {PROJECT_ROOT}")
print(f"📂 Images sources : {PATH_IMG_TRAIN}")
print(f"📂 Dossier de sortie (pour Oussama) : {OUTPUT_DIR}")

📂 Racine projet : C:\Users\amisf\Desktop\datascientest_projet
📂 Images sources : C:\Users\amisf\Desktop\datascientest_projet\images\images\image_train
📂 Dossier de sortie (pour Oussama) : C:\Users\amisf\Desktop\datascientest_projet\implementation\outputs


C:\Users\amisf\anaconda3\envs\masterclass_tooling\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [4]:


# --- 1. LE BON  CHEMINS ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"

# Chemin spécifique pour les fichiers CSV
PATH_CSV = os.path.join(PROJECT_ROOT, "data", "raw")

# Chemin spécifique pour les images (Mise à jour pour éviter la prochaine erreur)
PATH_IMG_TRAIN = os.path.join(PROJECT_ROOT, "data", "raw", "images", "images", "image_train")

print(f"📂 CSV recherchés dans : {PATH_CSV}")
print(f"📂 Images recherchées dans : {PATH_IMG_TRAIN}")

# --- 2. CHARGEMENT DES DONNÉES ---
try:
    df_x = pd.read_csv(os.path.join(PATH_CSV, "X_train_update.csv"), index_col=0)
    df_y = pd.read_csv(os.path.join(PATH_CSV, "Y_train_CVw08PX.csv"), index_col=0)
    df = pd.merge(df_x, df_y, left_index=True, right_index=True)
    print("✅ Fichiers CSV chargés avec succès.")
except FileNotFoundError as e:
    print(f"❌ Erreur : Impossible de trouver les fichiers dans {PATH_CSV}")
    raise e

# Construction du nom de fichier complet pour Keras
df['filename'] = df.apply(lambda x: f"image_{x['imageid']}_product_{x['productid']}.jpg", axis=1)
df['class_str'] = df['prdtypecode'].astype(str) # Keras demande des strings pour les classes

# --- 3. OVERSAMPLING CIBLÉ À 15 000 ---
TARGET_COUNT = 15000
df_balanced = pd.DataFrame()

print(f"⚖️ Démarrage du rééquilibrage (Cible : {TARGET_COUNT} images par classe)...")

for code in df['prdtypecode'].unique():
    df_class = df[df['prdtypecode'] == code]
    
    # On duplique les données (oversampling) pour atteindre 15 000
    df_resampled = resample(df_class, 
                            replace=True,     
                            n_samples=TARGET_COUNT, 
                            random_state=42)
    df_balanced = pd.concat([df_balanced, df_resampled])

# Mélange final
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ Dataset original : {len(df)} images.")
print(f"✅ Dataset augmenté : {len(df_balanced)} images prêt à l'emploi.")

📂 CSV recherchés dans : C:\Users\amisf\Desktop\datascientest_projet\data\raw
📂 Images recherchées dans : C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train
✅ Fichiers CSV chargés avec succès.
⚖️ Démarrage du rééquilibrage (Cible : 15000 images par classe)...
✅ Dataset original : 84916 images.
✅ Dataset augmenté : 405000 images prêt à l'emploi.


In [ ]:
# Définition de la taille d'entrée (Standard ResNet = 224x224)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Chargement de ResNet50 sans la tête de classification
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))

# Ajout du pooling pour aplatir les résultats
x = base_model.output
x = GlobalAveragePooling2D()(x)
model_extractor = Model(inputs=base_model.input, outputs=x)

print("🧠 Modèle ResNet50 chargé pour l'extraction de features.")
model_extractor.summary()

In [ ]:
# Configuration de l'Augmentation
# Ces paramètres seront appliqués à CHAQUE image passant dans le modèle
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Important pour ResNet
    rotation_range=30,      # Rotation aléatoire +/- 30°
    width_shift_range=0.2,  # Décalage horizontal
    height_shift_range=0.2, # Décalage vertical
    shear_range=0.1,        # Cisaillement
    zoom_range=0.2,         # Zoom aléatoire
    horizontal_flip=True,   # Miroir horizontal
    fill_mode='nearest'     # Remplissage des pixels créés
)

# Création du flux de données
# ATTENTION : shuffle=False est CRUCIAL ici pour garder l'ordre des labels !
generator = datagen.flow_from_dataframe(
    dataframe=df_balanced,
    directory=PATH_IMG_TRAIN,
    x_col="filename",
    y_col="class_str",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None, # On ne veut que les features, pas les labels pour l'instant
    shuffle=False    # IMPÉRATIF pour aligner X et y
)

In [ ]:
print("🚀 Début de l'extraction des features... Prenez un café ☕")

# Extraction (Prediction)
# Le résultat sera une matrice Numpy de taille (405000, 2048)
features = model_extractor.predict(generator, verbose=1)

print(f"✅ Extraction terminée. Dimensions des features : {features.shape}")

In [ ]:
# Récupération des labels correspondant aux features (grâce à shuffle=False)
# generator.classes nous donne les indices (0, 1, 2...)
# generator.class_indices nous donne le mapping {'10': 0, '1140': 1...}

labels_indices = generator.classes
class_indices_map = generator.class_indices
# Inversion du mapping pour retrouver les codes produits originaux (ex: 2583)
idx_to_code = {v: k for k, v in class_indices_map.items()}
labels_original_code = [int(idx_to_code[i]) for i in labels_indices]

# Conversion en Numpy Array
y_train_augmented = np.array(labels_original_code)

# Sauvegarde
print("💾 Sauvegarde des fichiers .npy...")

np.save(os.path.join(OUTPUT_DIR, 'train_features_resnet50_augmented.npy'), features)
np.save(os.path.join(OUTPUT_DIR, 'train_labels_augmented.npy'), y_train_augmented)

# Sauvegarde des métadonnées (Classes, Mapping)
import json
metadata = {
    'num_classes': 27,
    'feature_dim': 2048,
    'model_source': 'ResNet50_ImageNet_Augmented',
    'class_mapping': class_indices_map
}
with open(os.path.join(OUTPUT_DIR, 'metadata_augmented.json'), 'w') as f:
    json.dump(metadata, f)

print("✅ Tout est sauvegardé !")
print(f"   -> X (Features) : {os.path.join(OUTPUT_DIR, 'train_features_resnet50_augmented.npy')}")
print(f"   -> Y (Labels)   : {os.path.join(OUTPUT_DIR, 'train_labels_augmented.npy')}")